In [5]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data sources: CSO (House Prices, Wages), RTB (Rents)
# Note: 2025/2026 figures are based on latest available Q3/Q4 snapshots

2. The Dataset (2006–2026)

This block contains the historical data points for the national average/median figures

In [6]:
data = {
    'Year': list(range(2006, 2027)),
    'Avg_House_Price': [
        310000, 349838, 325000, 260000, 220000, 190000, 175000, 185000, 205000, 225000,
        240000, 265000, 290000, 305000, 310000, 335000, 360000, 375000, 395000, 415000, 435000
    ],
    'Avg_Monthly_Rent': [
        950, 1050, 990, 850, 800, 780, 750, 800, 880, 950,
        1050, 1150, 1250, 1350, 1400, 1450, 1550, 1680, 1750, 1820, 1900
    ],
    'Median_Annual_Wage': [
        31000, 32500, 33200, 32800, 32500, 32000, 32200, 32500, 33000, 33800,
        34500, 35500, 36800, 37500, 38500, 40000, 41823, 43221, 44816, 46500, 48000
    ]
}

df = pd.DataFrame(data)

3. Creating the Interactive Visualization

This code creates a dual-axis chart so you can compare House Prices (large numbers) with Rents and Wages (smaller numbers) simultaneously.

In [7]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add House Prices
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Avg_House_Price'], name="Avg House Price (€)",
               line=dict(color='firebrick', width=4)),
    secondary_y=False,
)

# Add Median Wage
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Median_Annual_Wage'], name="Median Annual Wage (€)",
               line=dict(color='royalblue', width=3, dash='dash')),
    secondary_y=False,
)

# Add Rent (Multiplied by 12 for annual comparison)
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Avg_Monthly_Rent'] * 12, name="Annual Rent Cost (€)",
               line=dict(color='orange', width=3)),
    secondary_y=False,
)

# Formatting
fig.update_layout(
    title='<b>Ireland Economic Trends: Housing vs Wages (2006-2026)</b>',
    xaxis_title='Year',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.update_yaxes(title_text="<b>Euro (€)</b>", secondary_y=False)

fig.show()

In [8]:
# Calculate Price-to-Income Ratio
# PTI = Avg House Price / Median Annual Wage
df['Price_to_Income_Ratio'] = df['Avg_House_Price'] / df['Median_Annual_Wage']

# Create a specialized chart for Affordability
fig_pti = go.Figure()

fig_pti.add_trace(go.Scatter(
    x=df['Year'],
    y=df['Price_to_Income_Ratio'],
    mode='lines+markers',
    name='PTI Ratio',
    line=dict(color='mediumpurple', width=4),
    hovertemplate='Year: %{x}<br>Ratio: %{y:.2f}x Annual Salary'
))

# Add a reference line for "Historical Sustainability" (often cited around 4.0 - 5.0)
fig_pti.add_hline(y=5.0, line_dash="dot", line_color="green",
              annotation_text="Higher Affordability Threshold", annotation_position="bottom right")

fig_pti.update_layout(
    title='<b>Housing Affordability: Price-to-Income Ratio (Ireland 2006-2026)</b>',
    xaxis_title='Year',
    yaxis_title='Years of Median Salary to Buy a Home',
    template='plotly_white',
    yaxis=dict(range=[0, 12])
)

fig_pti.show()

In [9]:
# 1. Calculate Monthly Income from our annual data
df['Median_Monthly_Wage'] = df['Median_Annual_Wage'] / 12

# 2. Calculate Rent as % of Monthly Income
df['Rent_Percent_of_Income'] = (df['Avg_Monthly_Rent'] / df['Median_Monthly_Wage']) * 100

# 3. Visualize the "Rental Burden"
fig_rent = go.Figure()

fig_rent.add_trace(go.Scatter(
    x=df['Year'],
    y=df['Rent_Percent_of_Income'],
    fill='tozeroy', # Shading the area for visual impact
    name='Rent Burden %',
    line=dict(color='darkorange', width=3),
    hovertemplate='Year: %{x}<br>Rent: %{y:.1f}% of Pay'
))

# Add the 30% "Affordability" benchmark
fig_rent.add_hline(y=30, line_dash="dash", line_color="red",
                  annotation_text="30% 'Affordability' Limit", annotation_position="top left")

fig_rent.update_layout(
    title='<b>The Rental Squeeze: Rent as % of Median Gross Monthly Income</b>',
    xaxis_title='Year',
    yaxis_title='Percentage of Monthly Pay (%)',
    template='plotly_white',
    yaxis=dict(range=[20, 55]) # Focus on the 20% to 55% range
)

fig_rent.show()